In [1]:
%load_ext autoreload
%autoreload

import sys
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tabulate import tabulate
from joblib import dump, load
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc

sys.path.append('../')
from LinearModels import display_plot_logreg, display_plot_svc
sys.path.append('../../utils')
from utils import feature_selection, thomas_parser, PCA_reduction, perf, time_comparison
from toBoolean import convert

### Time fiability


In [2]:
gt = pd.read_csv("../../../dumps/default_20190615_6000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_train = gt[cols]
target_train = gt['label']

scaler = Normalizer()
scaler.fit(data_train)
data_train = scaler.transform(data_train)

tree = LogisticRegression(C=0.01, max_iter=100,random_state=0)
tree.fit(data_train, target_train)
dump(tree,"snapshots/logreg_default_20190615_6000.joblib")

['snapshots/logreg_default_20190615_6000.joblib']

In [3]:
tree = load("snapshots/logreg_default_20190615_6000.joblib")

gt = pd.read_csv("../../../dumps/default_20190808_1000.csv")
cols = [col for col in gt.columns if col not in ['label']]
data_test = gt[cols]
target_test = gt['label']

data_test = scaler.transform(data_test)

print("Accuracy on training set: {:.3f}".format(tree.score(data_train, target_train))) 
print("Accuracy on test set: {:.3f}".format(tree.score(data_test, target_test)))

Accuracy on training set: 0.905
Accuracy on test set: 0.896


Conclusion (for both classifiers) : time does clearly impact on the learning of these two classifiers. We observe a drop of performance of around 10% which is quite significant.

#### Long run

In [4]:
time_comparison('log','../../')

Acceptation threshold : 1/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.7965         0.691
                       14000                   2               0.782857       0.691
                       21000                   3               0.754286       0.691
                       31000                   4.42857         0.690194       0.695
Acceptation threshold : 2/5 

  # malwares in training set    Approx. period in weeks    Training acc    Test acc
----------------------------  -------------------------  --------------  ----------
                        6000                   0.857143        0.868833       0.855
                       14000                   2               0.856286       0.855
                       21000                   3               0.827905       0.855
                

Increase the size of the training set doesn't improve the accuracies, it either stagnates or even decreases a bit. It's probably because the classifiers learn more informations that are spread around a lot of different values and therefore have more trouble to come out with good weights.